# Get ROI Time Series

In [ ]:
def roi_maskdims(img_shape, nROIs):
    if isinstance(img_shape, tuple):
        img_shape = list(img_shape)

    mask_shape = list(img_shape)
    mask_shape.append(nROIs)

    del(img_shape, nROIs)
    return(mask_shape)

In [ ]:
def maskROIs_as_time(mat):
    import numpy as np
    import nibabel as nib

    if isinstance(mat, str):
        mat = nib.load(mat)
        mat = img.get_fdata()

    roi_ls = np.unique(mat)
    roi_ls = roi_ls[roi_ls != 0]

    mat_dim = list(mat.shape)
    mat_dim.append(1)
    mat = np.reshape(mat, mat_dim)

    roi_mat = np.tile(mat, (1,1,1,len(roi_ls)))

    for ind,val in zip(range(len(roi_ls)), roi_ls):
        roi_mat[:,:,:,ind] = roi_mat[:,:,:,ind] - (val-1)
    
    roi_mat[roi_mat != 1] = 0

    del(mat, roi_ls, ind, val)
    return(roi_mat)


In [ ]:
def mult_NiiByMask(img, mask):
    # for each point along the fourth dimension of mat4d, element-wise multiply mat4d and mat3d
    import numpy as np

    if len(img.shape) != 4:
        raise Exception("img must be a 4-dimensional array")
    if len(mask.shape) != 3:
        raise Exception("mask must be a 3-dimensional array")

    mask_4d_dim = list(mask.shape)
    mask_4d_dim.append(1)
    
    mask = np.reshape(mask, mask_4d_dim)
    mask = np.tile(mask, (1,1,1,img.shape[3]))
    
    prod_mat = np.zeros(img.shape)
    prod_mat[np.nonzero(mask)] = img[np.nonzero(mask)]
    
    del(img, mask)
    return(prod_mat) # output is x,y,z,time


In [ ]:
def roi_tcourse(img, mask):
    # compute the mean time course for each ROI 
    import numpy as np

    if img.shape[0:3] != mask.shape:
        raise Exception("img and mask are not in the same 3d space.")

    mean_tcourse = img / np.sum(mask)
    mean_tcourse = np.sum(mean_tcourse, (0,1,2))

    del(img, mask)
    return(mean_tcourse)

In [ ]:
def atlas_tcourse(nifti_path, mask_path, file_out=None):
    import numpy as np
    import nibabel as nib

    print("+ IMPORTING DATA")
    img = nib.load(nifti_path)
    img = img.get_fdata()
    
    mask = nib.load(mask_path)
    mask = mask.get_fdata()

    print("+ SPLITTING ROIs")
    mask = maskROIs_as_time(mask) # mask becomes 4d

    if len(img.shape) != 4:
        raise Exception("The provided nifti file must be 4D.")
    if img.shape[0:3] != mask.shape[0:3]:
        raise Exception("The provided nifti file and mask must be in the same 3D space.")
    
    print("+ COMPUTING MEAN TIME COURSE")
    mean_tcourse = np.zeros([img.shape[3], mask.shape[3]])
    
    for roi in range(mask.shape[3]):
        prod_mat = mult_NiiByMask(img, mask[:,:,:,roi]) # output is x,y,z,time
        mean_tcourse[:,roi] = roi_tcourse(prod_mat, mask[:,:,:,roi])
        
        del(prod_mat)

    print("+ SAVING")
    if isinstance(file_out,str):
        hdr = ",".join(["roi_"+str(roi+1) for roi in range(mask.shape[3])])
        np.savetxt(file_out, mean_tcourse, header=hdr, comments='', delimiter=',')

    return(mean_tcourse)

# Sliding Time Window

In [ ]:
def n_twin(n_tpts, win_sz, step_sz, fill_tline=False):
    from math import floor
    
    num_twin = (n_tpts - win_sz) / step_sz
    
    if fill_tline:
        ovr_spill = num_twin.is_integer()
    else:
        ovr_spill = False

    return(floor(num_twin), ovr_spill) # returns the number of whole time windows and if there is spill-over


In [47]:
def sliding_twin(ttab, win_sz, step_sz, save_path=None, fill_tline=False):
    import numpy as np

    if len(ttab.shape) != 2:
        raise Exception("The table must be a 2-D array")

    num_twin, ovr_spill = n_twin(ttab.shape[0], win_sz, step_sz, fill_tline)

    fc_mats = np.zeros([ttab.shape[1], ttab.shape[1], num_twin+ovr_spill])

    win_pos = 0
    for win in range(num_twin):
        ttab_win = ttab[range(win_pos, win_pos+step_sz),:]
        fc_mats[:,:,win] = np.corrcoef(np.transpose(ttab_win))
        win_pos += step_sz

    if ovr_spill:
        ttab_win = ttab[win_pos:-1,:]
        fc_mats[:,:,num_twin+ovr_spill] = np.corrcoef(np.transpose(ttab_win))

    if save_path is not None:
        import scipy.io as sio

        sio.savemat(save_path, {"fc_mats":fc_mats})

    return(fc_mats)

# K-Means Clustering

In [52]:
def mat3Dto2D(mat_3d, order="C"):
    import numpy as np

    if len(mat_3d.shape) != 3:
        raise Exception("mat_3d must be a 3D array")
    
    new_shape = [mat_3d.shape[0]*mat_3d.shape[1], mat_3d.shape[2]]
    mat_2d    = np.reshape(mat_3d, new_shape, order)
    
    return(mat_2d)


In [69]:
def mat2Dto3D(mat_2d, order="C"):
    import numpy as np
    from math import sqrt

    if len(mat_2d.shape) != 2:
        raise Exception("mat_2d must be a 2D array")

    new_shape = [int(sqrt(mat_2d.shape[0])), int(sqrt(mat_2d.shape[0])), mat_2d.shape[1]]
    mat_3d    = np.reshape(mat_2d, new_shape, order)
    mat_3d    = np.transpose(mat_3d)

    return(mat_3d)


In [97]:
def find_knee(k_ls, vals):
    from kneed import KneeLocator

    kl = KneeLocator(
        k_ls, vals, curve="convex", direction="decreasing"
    )

    return(kl.elbow)

In [115]:
def det_k(mat, n_clusters=[2,3,4], n_init=10, max_iter=300, random_state=42):
    from numpy import argmin
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score

    if len(n_clusters) <= 1:
        raise Exception("n_clusters must be a list with at least 2 entries (i.e., n > 1)")
    if 1 in n_clusters:
        raise Exception("n_clusters cannot contain 1.")

    kmeans_kwargs = {
            "init": "random",
            "n_init": n_init,
            "max_iter": max_iter,
            "random_state": random_state,
        }

    # A list holds the SSE values for each k
    sse  = []
    silh = []
    for k in n_clusters:
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(mat)
        # sse knee
        sse.append(kmeans.inertia_)
        # silhouette coefficient
        score = silhouette_score(mat, kmeans.labels_)
        silh.append(score)
    
    knee = find_knee(n_clusters, sse)

    out_vals = {
        "SSE": sse,
        "Silhouette": silh
    }

    return(knee, out_vals)

    

In [125]:
def kmeans_fc(mat, n_clusters=3, n_init=10, max_iter=300, random_state=42):
    import numpy as np
    from sklearn.cluster import KMeans

    if len(mat.shape) == 3:
        mat = mat3Dto2D(mat)
    elif len(mat.shape) != 2:
        raise Exception("The provided matrix to cluster must be either 2D (obs,  var) or 3D (varb, varb, obs).")

    kmeans = KMeans(
        init="random",
        n_clusters=n_clusters,
        n_init=n_init,
        max_iter=max_iter,
        random_state=random_state
    )

    clust  = kmeans.fit(mat)

    return(clust)

# Testing Area

In [ ]:
# mask_path  = "/mnt/c/Users/John/Desktop/MNI/MNI-maxprob-thr0-2mm.nii.gz"
# nifti_path = "/mnt/d/Downloads/ds000031-download/sub-01/ses-003/out/func/scrub_motreg_s_nl_m_t_func.nii.gz"
# file_out   = "/mnt/c/Users/John/Desktop/mean_roi_tcourse.csv"

mask_path  = "C:/Users/John/Desktop/MNI/MNI-maxprob-thr0-2mm.nii.gz"
nifti_path = "D:/Downloads/ds000031-download/sub-01/ses-003/out/func/scrub_motreg_s_nl_m_t_func.nii.gz"
file_out   = "C:/Users/John/Desktop/mean_roi_tcourse.csv"

tcourse = atlas_tcourse(nifti_path, mask_path, file_out)

In [38]:
file_out   = "C:/Users/John/Desktop/mean_roi_tcourse.csv"

mat = np.loadtxt(open(file_out, "rb"), delimiter=",", skiprows=1)

In [48]:
fc_mats = sliding_twin(mat, 30, 10, save_path="C:/Users/John/Desktop/test_FCmat.mat", fill_tline=False)

In [71]:
test2d=mat3Dto2D(fc_mats, order="C")
test3d=mat2Dto3D(test2d , order="C")


In [116]:
knee, vals,  = det_k(test2d, n_clusters=[2,3,4,5,6,7,8,9,10], n_init=10, max_iter=500, random_state=42)

In [126]:
clusters = kmeans_fc(fc_mats, n_clusters=knee, n_init=20, max_iter=500, random_state=42)

array([2, 1, 2, 2, 0, 0, 2, 0, 2, 1, 2, 1, 1, 3, 3, 1, 3, 1, 2, 1, 2, 2,
       0, 0, 4, 0, 2, 2, 1, 2, 2, 0, 0, 2, 0, 2, 0, 3, 0, 0, 2, 2, 4, 2,
       0, 0, 3, 0, 0, 2, 2, 4, 2, 0, 2, 1, 4, 2, 4, 4, 2, 4, 4, 0, 3, 0,
       0, 2, 2, 4, 2, 0, 2, 1, 2, 2, 0, 0, 4, 0, 2])